## Reference states

In this lesson, we'll explore how we can initialize our system with a *reference state* to help our variational algorithm converge faster. First, we'll learn how to construct a reference state by hand, and then explore several standard options that can be used in a variational algorithm.

![Reference States](reference_states.png)

## Default state

A *reference state* $|\rho\rangle$ refers to the initial, fixed start for our problem. To prepare a reference state, we will need to apply the appropriate unitary $U_R$ at the start of our quantum circuit, such that $|\rho\rangle = U_R |0\rangle$. If you have an educated guess or datapoint from an existing optimal solution, the variational algorithm will likely converge faster if you start with that _initial state_.

The simplest possible reference state is the _default state_, where we use the starting state of an $n$-qubit quantum circuit: $|0\rangle^{\otimes n}$. For the default state, our unitary operator $U_R \equiv I$. Due to its simplicity, the default state is a valid reference state used in many scenarios.

## Classical reference state

Imagine that you have a $3$-qubit system and you want to start in the state $|001\rangle$ instead of $|000\rangle$. This is a purely classical reference state and, in order to build it, you simply need to apply an [X gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.XGate.html) to qubit $0$ (following Qiskit's qubit ordering), since $|001\rangle=X_0|000\rangle$. 

In this case, our unitary operator $U_R \equiv X_0$, and leads to the reference state $|\rho\rangle \equiv |001\rangle$.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(3)
qc.x(0)

print(Statevector(qc))
qc.draw("mpl")

## Quantum reference state

Now imagine that you want to start with a more complicated state with some superposition and/or entanglement, like $\frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$. 

One way to get this state from $|000\rangle$ is to apply a [Hadamard gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.HGate.html) to qubit $0$ ($H_0$), a [CNOT (CX)](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) whose control qubit is qubit $0$ and whose target qubit is qubit $1$ ($CNOT_{01}$) and finally an $X$ gate acting on qubit $2$ ($X_2$). 

In this case, our unitary is $U_{R} \equiv X_2CNOT_{01}H_0|000\rangle$, and our reference state $|\rho\rangle \equiv \frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$ 

In [ ]:
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0,1)
qc.x(2)

print(Statevector(qc))
qc.draw("mpl")

## Application-specific reference states

### Quantum Chemistry

In Quantum Chemistry, the _Hartree-Fock_ state is an approximation of the ground state of an atom or molecule. If our goal was to create a variational algorithm to find the exact ground state, this known classical approximation can be encoded as a reference state to help our algorithm converge faster. In this example, we generate an Electronic Structure Problem for $H_{2}$, and use the problem's `num_spatial_orbitals`, `num_particles` to create our initial state.

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

converter = QubitConverter(JordanWignerMapper())

h2_reference_state = HartreeFock(
    h2_problem.num_spatial_orbitals,
    h2_problem.num_particles,
    converter
)

h2_reference_state.decompose().draw('mpl')

### Quantum Machine Learning

In a [variational quantum classifier (VQC)](https://learn.qiskit.org/course/machine-learning/variational-classification), training data is encoded into a quantum state with a parameterized circuit called _feature map_, where each value of the parameters represents a data point from the training dataset. An example of this kind of circuits is Qiskit's [ZFeatureMap](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZFeatureMap.html)

In [ ]:
from qiskit.circuit.library import ZFeatureMap

z_feature_map_reference = ZFeatureMap(3, reps=3, insert_barriers=True)
z_feature_map_reference.decompose().draw('mpl')

With this lesson, you learned how to initialize your system using:

- Default reference state
- Classical reference states
- Quantum reference states
- Application-specific reference states

Our high-level variational circuit looks as follows:

![Reference State Circuit](circuit_reference_state.png)

While reference states are fixed, initial starting points, we can use an *ansatz* to define a *variational form* to represent a collection of parametrized states for our variational algorithm to explore.